In [9]:
import torch
from torch import nn
 
# 定义网络模型
class LeNet5(nn.Module):
    #初始化网络
    def __init__(self):
        super(LeNet5, self).__init__()
        
        self.c1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=5, padding=2)        
        self.Sigmoid = nn.Sigmoid()
        self.s2 = nn.AvgPool2d(kernel_size=2, stride=2)       
        self.c3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5)
        self.s4 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.c5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5)
        
        self.flatten = nn.Flatten()
        self.f6 = nn.Linear(in_features=256, out_features=512)
        self.output = nn.Linear(in_features=512, out_features=10)
        
    def forward(self, x):
        x = self.Sigmoid(self.c1(x))
        x = self.s2(x)
        x = self.Sigmoid(self.c3(x))
        x = self.s4(x)
        x = self.c5(x)
        x = self.flatten(x)
        x = self.f6(x)
        x = self.output(x)
        return x
x = torch.rand([1, 1, 28, 28])
model = LeNet5()
y = model(x)

In [10]:
import torch
from torch import nn
from torch.optim import lr_scheduler
from torchvision import datasets, transforms
import os
 
#数据转化为tensor格式
data_transform = transforms.Compose([
    transforms.ToTensor()
])
 
#加载训练数据集
train_dataset = datasets.MNIST(root='./data', train=True, transform=data_transform, download=True)
train_dataloader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=16, shuffle=True)
 
#加载测试数据集
test_dataset = datasets.MNIST(root='./data', train=False, transform=data_transform, download=True)
test_dataloader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size=16, shuffle=True)
 
#如果有显卡可以转到GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
 
#调用net里定义的网络模型LeNet5 将数据模型转到GPU
model = LeNet5().to(device)
 
#定义损失函数
loss_fn = nn.CrossEntropyLoss()
 
#定义优化器
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
 
#学习率每隔10轮变为原来的0.1
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
 
#定义训练函数
def train(dataloader, model, loss_fn, optimizer):
    loss, current, n = 0.0, 0.0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        output = model(X)
        cur_loss = loss_fn(output,y)
        _, pred = torch.max(output, axis=1)
        
        cur_acc = torch.sum(y == pred)/output.shape[0]
        
        optimizer.zero_grad()
        cur_loss.backward()
        optimizer.step()
        
        loss += cur_loss.item()
        current += cur_acc.item()
        n = n + 1
    print("train_loss  " + str(loss/n))
    print("train_acc  " + str(current/n))
 
def val(dataloader, model, loss_fn):
    model.eval()
    loss, current, n = 0.0, 0.0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            output = model(X)
            cur_loss = loss_fn(output,y)
            _, pred = torch.max(output, axis=1)           
            cur_acc = torch.sum(y == pred)/output.shape[0]          
            loss += cur_loss.item()
            current += cur_acc.item()
            n = n + 1
        print("test_loss  " + str(loss/n))
        print("test_acc  " + str(current/n))   
        
        return current/n
        
#开始训练
epoch = 20
min_acc = 0
 
for t in range(epoch):
    print(f'epoch{t+1}\n-----------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    a = val(test_dataloader, model, loss_fn)
    
    #保存最优模型
    if a > min_acc:
        folder = 'save_model'
        if not os.path.exists(folder):
            os.mkdir('save_model')
        min_acc = a
        print('save best model!!!!!!')
        torch.save(model.state_dict(), 'save_model/best_model.pth')
print('OK!!!!!!!!!!!!!!!!!')

epoch1
-----------------------------------
train_loss  2.3011375446955364
train_acc  0.11778333333333334
test_loss  2.2816249340057375
test_acc  0.1423
save best model!!!!!!
epoch2
-----------------------------------
train_loss  1.3250462160209815
train_acc  0.55395
test_loss  0.5080360377192498
test_acc  0.8458
save best model!!!!!!
epoch3
-----------------------------------
train_loss  0.44101752282083034
train_acc  0.8656
test_loss  0.3593500443041325
test_acc  0.8943
save best model!!!!!!
epoch4
-----------------------------------
train_loss  0.3498826849584778
train_acc  0.8937333333333334
test_loss  0.31659866755604743
test_acc  0.9067
save best model!!!!!!
epoch5
-----------------------------------
train_loss  0.29336957574958605
train_acc  0.9117333333333333
test_loss  0.24674546437859535
test_acc  0.925
save best model!!!!!!
epoch6
-----------------------------------
train_loss  0.24199639877850812
train_acc  0.9275833333333333
test_loss  0.20428638576492666
test_acc  0.9397
s

In [16]:
import torch
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.transforms import ToPILImage
 
#数据转化为tensor格式
data_transform = transforms.Compose([
    transforms.ToTensor()
])
 
#加载训练数据集
train_dataset = datasets.MNIST(root='./data', train=True, transform=data_transform, download=True)
train_dataloader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=16, shuffle=True)
 
#加载测试数据集
test_dataset = datasets.MNIST(root='./data', train=False, transform=data_transform, download=True)
test_dataloader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size=16, shuffle=True)
 
#如果有显卡可以转到GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
 
#调用net里定义的网络模型LeNet5 将数据模型转到GPU
model = LeNet5().to(device)
 
model.load_state_dict(torch.load(r"D:\Study Software\Jupyter\计算机视觉与应用实践作业\作业二\save_model\best_model.pth"))
 
#获取结果
classes = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
]
 
#把tensor转化为图片，方便可视化
show = ToPILImage()
 
#验证
print(len(test_dataset))
acc = 0
fal = 0
for i in range(len(test_dataset)//10):
    X, y = test_dataset[i][0], test_dataset[i][1]
#    show(X).show()
    X = Variable(torch.unsqueeze(X, dim=0).float(), requires_grad=False).to(device)  
    with torch.no_grad():
        pred = model(X)
        predicted, actual = classes[torch.argmax(pred[0])], classes[y]
        if predicted == actual:
            acc += 1
        else:
            fal += 1
        print(f'figure {str(i)}: predicted: "{predicted}", actual: "{actual}"')
print(f'figurenums {acc+fal}: acc: "{acc/(acc+fal)}"')

10000
figure 0: predicted: "7", actual: "7"
figure 1: predicted: "2", actual: "2"
figure 2: predicted: "1", actual: "1"
figure 3: predicted: "0", actual: "0"
figure 4: predicted: "4", actual: "4"
figure 5: predicted: "1", actual: "1"
figure 6: predicted: "4", actual: "4"
figure 7: predicted: "9", actual: "9"
figure 8: predicted: "5", actual: "5"
figure 9: predicted: "9", actual: "9"
figure 10: predicted: "0", actual: "0"
figure 11: predicted: "6", actual: "6"
figure 12: predicted: "9", actual: "9"
figure 13: predicted: "0", actual: "0"
figure 14: predicted: "1", actual: "1"
figure 15: predicted: "5", actual: "5"
figure 16: predicted: "9", actual: "9"
figure 17: predicted: "7", actual: "7"
figure 18: predicted: "3", actual: "3"
figure 19: predicted: "4", actual: "4"
figure 20: predicted: "9", actual: "9"
figure 21: predicted: "6", actual: "6"
figure 22: predicted: "6", actual: "6"
figure 23: predicted: "5", actual: "5"
figure 24: predicted: "4", actual: "4"
figure 25: predicted: "0", ac

figure 244: predicted: "2", actual: "2"
figure 245: predicted: "3", actual: "3"
figure 246: predicted: "0", actual: "0"
figure 247: predicted: "2", actual: "4"
figure 248: predicted: "4", actual: "4"
figure 249: predicted: "2", actual: "2"
figure 250: predicted: "4", actual: "4"
figure 251: predicted: "1", actual: "1"
figure 252: predicted: "9", actual: "9"
figure 253: predicted: "5", actual: "5"
figure 254: predicted: "7", actual: "7"
figure 255: predicted: "7", actual: "7"
figure 256: predicted: "2", actual: "2"
figure 257: predicted: "8", actual: "8"
figure 258: predicted: "2", actual: "2"
figure 259: predicted: "6", actual: "6"
figure 260: predicted: "8", actual: "8"
figure 261: predicted: "5", actual: "5"
figure 262: predicted: "7", actual: "7"
figure 263: predicted: "7", actual: "7"
figure 264: predicted: "9", actual: "9"
figure 265: predicted: "1", actual: "1"
figure 266: predicted: "8", actual: "8"
figure 267: predicted: "1", actual: "1"
figure 268: predicted: "8", actual: "8"


figure 467: predicted: "2", actual: "2"
figure 468: predicted: "7", actual: "7"
figure 469: predicted: "5", actual: "5"
figure 470: predicted: "8", actual: "8"
figure 471: predicted: "9", actual: "9"
figure 472: predicted: "6", actual: "6"
figure 473: predicted: "1", actual: "1"
figure 474: predicted: "8", actual: "8"
figure 475: predicted: "4", actual: "4"
figure 476: predicted: "1", actual: "1"
figure 477: predicted: "2", actual: "2"
figure 478: predicted: "5", actual: "5"
figure 479: predicted: "9", actual: "9"
figure 480: predicted: "1", actual: "1"
figure 481: predicted: "9", actual: "9"
figure 482: predicted: "7", actual: "7"
figure 483: predicted: "5", actual: "5"
figure 484: predicted: "4", actual: "4"
figure 485: predicted: "0", actual: "0"
figure 486: predicted: "8", actual: "8"
figure 487: predicted: "9", actual: "9"
figure 488: predicted: "9", actual: "9"
figure 489: predicted: "1", actual: "1"
figure 490: predicted: "0", actual: "0"
figure 491: predicted: "5", actual: "5"


figure 717: predicted: "0", actual: "0"
figure 718: predicted: "8", actual: "8"
figure 719: predicted: "4", actual: "4"
figure 720: predicted: "8", actual: "5"
figure 721: predicted: "0", actual: "0"
figure 722: predicted: "4", actual: "4"
figure 723: predicted: "0", actual: "0"
figure 724: predicted: "6", actual: "6"
figure 725: predicted: "1", actual: "1"
figure 726: predicted: "7", actual: "7"
figure 727: predicted: "3", actual: "3"
figure 728: predicted: "2", actual: "2"
figure 729: predicted: "6", actual: "6"
figure 730: predicted: "7", actual: "7"
figure 731: predicted: "2", actual: "2"
figure 732: predicted: "6", actual: "6"
figure 733: predicted: "9", actual: "9"
figure 734: predicted: "3", actual: "3"
figure 735: predicted: "1", actual: "1"
figure 736: predicted: "4", actual: "4"
figure 737: predicted: "6", actual: "6"
figure 738: predicted: "2", actual: "2"
figure 739: predicted: "5", actual: "5"
figure 740: predicted: "9", actual: "4"
figure 741: predicted: "2", actual: "2"


figure 971: predicted: "4", actual: "4"
figure 972: predicted: "0", actual: "0"
figure 973: predicted: "6", actual: "6"
figure 974: predicted: "2", actual: "2"
figure 975: predicted: "2", actual: "2"
figure 976: predicted: "3", actual: "3"
figure 977: predicted: "1", actual: "1"
figure 978: predicted: "5", actual: "5"
figure 979: predicted: "1", actual: "1"
figure 980: predicted: "2", actual: "2"
figure 981: predicted: "0", actual: "0"
figure 982: predicted: "3", actual: "3"
figure 983: predicted: "8", actual: "8"
figure 984: predicted: "1", actual: "1"
figure 985: predicted: "2", actual: "2"
figure 986: predicted: "6", actual: "6"
figure 987: predicted: "7", actual: "7"
figure 988: predicted: "1", actual: "1"
figure 989: predicted: "6", actual: "6"
figure 990: predicted: "2", actual: "2"
figure 991: predicted: "3", actual: "3"
figure 992: predicted: "9", actual: "9"
figure 993: predicted: "0", actual: "0"
figure 994: predicted: "1", actual: "1"
figure 995: predicted: "2", actual: "2"
